# How *spock* works

General note: If you wish to update the number of hours observe on the TRAPPIST telescopes as well you need to have acces to the **Hours_observation_TS_TN** google sheet. For this contact Elsa Ducrot (educrot@uliege.be)

## How `spock.long_term_scheduler` works

The SPECULOOS target list was contructed as described in [*Sebastian et al. 2020*](http://arxiv.org/abs/2011.02069) and include 1657 photometrically classified late-type dwarfs. These targets are divided in three distinct programs:
* Program 1: 365 dwarfs that are small and nearby enough to make possible the detailed atmospheric characterisation of an "Earth-like" planet with the upcoming JWST
* Program 2: 171 dwarfs of M5-type and later for which a significant detection of a planet similar to TRAPPIST-1b should be within reach of TESS
* Program 3: 1121 dwarfs later than M6-type that aims to perform a statistical census of short-period planets around ultracool dwarf stars.

## How `spock.short_term_scheduler` works

The aim of the `short_term_scheduler` is to modify existing plan by inserting a new block (corresponding to a *special* or *follow-up* target) 

<img controls src="../../../_static/css/situations_AB.png" width=600/>

### Situation A

<img src="../../../_static/css/situation_A1.png" width=300/>

<img src="../../../_static/css/situation_A2.png" width=300/>

<img src="../../../_static/css/situation_A3.png" width=300/>

<img src="../../../_static/css/situation_A4.png" width=300/>

### Situation B

![title](../../../_static/css/situation_B1.png)

<img src="../../../_static/css/situation_B2.png" width=300/>

<img src="../../../_static/css/situation_B3.png" width=300/>